In [81]:
import pickle
import networkx as nx
from networkx.algorithms import community
import matplotlib.pyplot as plt
import random
import numpy as np
from time import time
%matplotlib inline

In [5]:
INTERSECTION = 0
BRIDGE = 1

In [7]:
# Switch out of NetworkX 1
G = nx.read_gpickle("quick_traffic_model/input/graphMTC_CentroidsLength3int.gpickle")
pickle.dump([G.nodes(data=True), G.edges(data=True)], open('quick_traffic_model/input/graphMTC_CentroidsLength3int_nx2.pickle', 'wb'))  

In [4]:
# Switch into NetworkX 2
nodes, edges = pickle.load(open('quick_traffic_model/input/graphMTC_CentroidsLength3int_nx2.pickle', 'r'))  
G = nx.DiGraph()  
G.add_nodes_from(nodes)  
G.add_edges_from(edges) 
nx.write_gpickle(G, "quick_traffic_model/input/graphMTC_CentroidsLength3int_nx2.gpickle")

In [6]:
# Load road network
H = nx.read_gpickle("quick_traffic_model/input/graphMTC_CentroidsLength3int_nx2.gpickle")
G = nx.Graph()
G.add_nodes_from(H.nodes(data=True))
G.add_edges_from(H.edges(data=True))

# Make intersections node type 0 to separate from bridges
for n in G.nodes():
    G.nodes[n]['nodetype'] = INTERSECTION # intersection, not bridge

print 'finished loading road network'
G = G.to_undirected()

# Open bridge database
bridges_dict = None
with open('quick_traffic_model/input/20140114_master_bridge_dict.pkl','rb') as f:
    bridges_dict = pickle.load(f)

# Add list of bridges on each road segment
for b_num in bridges_dict.keys():
    bridge_edges = bridges_dict[b_num]['a_b_pairs_direct']
    new_id = str(bridges_dict[b_num]['new_id'])
    #if len(bridge_edges) == 0:
    #    print bridges_dict[b_num]
    for [u,v] in bridge_edges:
        G.edges[str(u),str(v)]['bridges'].append('b' + new_id)
        
intersections_G = G.copy()

# For each bridge, make a new node connecting to the proper intersections
for (u,v) in intersections_G.edges():
    if len(intersections_G.edges[u,v]['bridges']) > 0:
        bridge_list = G.edges[u,v]['bridges']
        for bridge in bridge_list:
            if not G.has_node(bridge):
                G.add_node(bridge, nodetype=BRIDGE)
            G.add_edge(u, bridge)
            G.add_edge(bridge, v)
        G.remove_edge(u,v)

print 'finished adding bridges'
bridges_and_intersections_G = G.copy()

# Remove the intersections
for n in bridges_and_intersections_G.nodes():
    if G.nodes[n]['nodetype'] == INTERSECTION:
        for v1n in G.neighbors(n):
            for v2n in G.neighbors(n):
                if G.nodes[v1n]['nodetype'] == BRIDGE and G.nodes[v2n]['nodetype'] == BRIDGE:
                    G.add_edge(v1n, v2n)
        G.remove_node(n)

print 'finished removing road intersections'
bridges_G = G.copy()

finished loading road network
finished adding bridges
finished removing road intersections


In [ ]:
#print G.nodes()
#print G.edges()
print 'INTERSECTIONS'
for (u,v) in intersections_G.edges():
    if len(intersections_G.get_edge_data(u, v)['bridges']) > 1:
        print u,v,intersections_G.get_edge_data(u, v)
    
# 7016 6970 {'distance': 2.079999, 'dailyvolume': 133477, 'capacity_0': 9500, 'lanes': 5, 
# 'bridges': ['b956', 'b960', 'b1743'], 't_a': 124, 'flow': 0, 'distance_0': 2.079999, 'capacity': 9500, 't_0': 124}

    
#print 'INTERSECTIONS AND BRIDGES'
#for (u,v) in intersections_G.edges()[:100]:
#    print u,v,bridges_and_intersections_G.get_edge_data(u, v)

In [22]:
# all on same road segment
print 'b956', 'b960', bridges_G.get_edge_data('b956', 'b960')
print 'b1743', 'b960', bridges_G.get_edge_data('b1743', 'b960')
print 'b956', 'b1743', bridges_G.get_edge_data('b956', 'b1743')

# b959 is adjacent to all 3 if in parallel, not if in series
print 'b959', 'b956', bridges_G.get_edge_data('b956', 'b960')
print 'b959', 'b960', bridges_G.get_edge_data('b1743', 'b960')
print 'b959', 'b1743', bridges_G.get_edge_data('b956', 'b1743')

# confirm that gives different results than missing edges
print 'fake', 'fake2', bridges_G.get_edge_data('fake', 'fake2')

b956 b960 {}
b1743 b960 {}
b956 b1743 {}
b959 b956 {}
b959 b960 {}
b959 b1743 {}
fake fake2 None


In [24]:
compare_community_metrics(bridges_G)

(0.0020441192728135136, 0.010114347459539048, 0.9979474548440066)
(0.0020423498373350613, 0.010111181036291168, 0.9975369458128078)
(0.002039379713487124, 0.01010643140141935, 0.9969211822660099)
(0.0020489852203699624, 0.010125429940906627, 0.999384236453202)
(0.0020252452943528426, 0.010089016073556011, 0.9946633825944171)
(0.002025245294341641, 0.010089016073556011, 0.9946633825944171)
(0.0020478055967213865, 0.010122263517658747, 0.9989737274220033)
(0.001997755850279498, 0.010065267899196916, 0.9915845648604269)
(0.0020484164732534646, 0.010123846729282687, 0.9991789819376026)
(0.0020171353817492617, 0.010081100015436313, 0.9936371100164204)
('modularity:', array([7, 9, 5, 4, 2, 1, 0, 6, 8, 3]))
('performance:', array([7, 9, 4, 5, 2, 1, 0, 6, 8, 3]))
('coverage:', array([7, 9, 4, 5, 2, 1, 0, 6, 8, 3]))


In [107]:
def compare_community_metrics(graph, num_iters=10, num_retrofits=5):
    modularity = np.zeros(num_iters)
    performance = np.zeros(num_iters)
    coverage = np.zeros(num_iters)
    
    for i in range(num_iters):
        retrofitted = random.sample(graph.nodes(), num_retrofits)
        not_retrofitted = [n for n in graph.nodes() if n not in retrofitted]
        modularity[i] = community.modularity(graph, [retrofitted, not_retrofitted])
        performance[i] = community.performance(graph, [retrofitted, not_retrofitted])
        coverage[i] = community.coverage(graph, [retrofitted, not_retrofitted])
        print(modularity[i], performance[i], coverage[i])
    
    print('modularity:', np.argsort(modularity)) 
    print('performance:', np.argsort(performance))
    print('coverage:', np.argsort(coverage))

def get_highest_coverage(graph, num_tries=100, num_retrofits=5, num_samples=10):
    # run num_tries simulations of retrofitting num_retrofits bridges, and pick the best num_samples
    retrofit_sets = []
    coverages = np.zeros(num_tries)
    
    for i in range(num_tries):
        # pick num_retrofits nodes to be retrofitted
        retrofitted = random.sample(graph.nodes(), num_retrofits)
        not_retrofitted = [n for n in graph.nodes() if n not in retrofitted]
        #new_part = [retrofitted, not_retrofitted]
        new_part = community.kernighan_lin_bisection(graph, partition=[retrofitted, not_retrofitted], max_iter=2)
        coverages[i] = community.coverage(graph, new_part)
        retrofit_sets.append(list(new_part[0]))
        print i
    
    argsorted = np.argsort(coverages)
    selection = [retrofit_sets[i] for i in argsorted[:num_samples]]
    return selection

In [109]:
bridges_G.remove_edges_from(bridges_G.selfloop_edges())
bridges_G.remove_nodes_from(list(nx.isolates(bridges_G)) )
tic = time()
retrofit_samples = get_highest_coverage(bridges_G, num_tries=10, num_retrofits=50, num_samples=10)
print(time() - tic)
print retrofit_samples

0
1
2
3
4
5
6
7
8
9
112.386660099
[['b828', 'b543', 'b786', 'b666', 'b465', 'b620', 'b626', 'b1125', 'b818', 'b183', 'b838', 'b1141', 'b1143', 'b1462', 'b1180', 'b1461', 'b570', 'b1186', 'b675', 'b536', 'b537', 'b1222', 'b638', 'b1226', 'b1340', 'b1135', 'b1455', 'b1200', 'b806', 'b1412', 'b134', 'b7', 'b562', 'b407', 'b1396', 'b683', 'b602', 'b1028', 'b741', 'b76', 'b79', 'b1165', 'b1164', 'b552', 'b559', 'b790', 'b690', 'b712', 'b597', 'b453'], ['b949', 'b948', 'b1635', 'b1479', 'b1431', 'b343', 'b1470', 'b154', 'b1464', 'b509', 'b112', 'b504', 'b1642', 'b465', 'b1646', 'b1481', 'b951', 'b1708', 'b1507', 'b230', 'b1547', 'b144', 'b143', 'b1388', 'b904', 'b431', 'b805', 'b1410', 'b482', 'b407', 'b177', 'b14', 'b448', 'b1664', 'b1442', 'b1372', 'b30', 'b31', 'b33', 'b990', 'b992', 'b993', 'b457', 'b387', 'b698', 'b165', 'b167', 'b922', 'b297', 'b453'], ['b194', 'b1576', 'b825', 'b826', 'b820', 'b1635', 'b1642', 'b386', 'b428', 'b465', 'b1646', 'b1482', 'b1251', 'b833', 'b57', 'b333', '

In [106]:
for sample in retrofit_samples:
    for b in sample:
        print b
        for n in bridges_G.neighbors(b):
            print '\t', n
    print '\n'

b1037
	b1095
	b1075
	b1074
	b1050
	b1070
	b1055
	b1072
	b815
	b1088
	b817
	b1047
	b1043
	b767
	b1049
	b1081
	b1084
	b1085
	b1086
	b1069
	b1066
	b1065
	b1062
	b1063
	b1061
	b1054
	b798
	b1048
	b1117
b1120
	b1113
	b1317
	b1122
	b822
b52
	b396
	b397
	b395
	b389
	b375
	b376
	b354
	b355
	b345
	b359
	b288
	b284
	b412
b1330
	b1323
	b1306
b1142
	b1166
	b1300
	b1167
	b1320
	b1158
	b1159
	b1163
	b1157
	b1161
	b1152
	b1207
	b1313
	b1175
	b1176
	b1168
	b1170
	b1172
	b1162
b140
	b146
	b110
	b137
	b122
	b126
b1436
	b1435
	b1473
	b1446
b658
	b667
	b665
	b662
	b663
	b660
	b661
	b668
	b659
	b612
	b653
	b572
	b528
b1028
	b806
b1098
	b1094
	b1093
	b1092
	b1091
	b1087
	b1089
	b1090
	b1099
	b1109
	b1108
b1717
	b1520
b778
	b1080
	b1058
	b792
	b793
	b795
	b779
	b769
	b755
	b780
	b770
	b771
	b772
	b773
b1292
	b1301
	b1293
	b1177
b759
	b744
	b678
	b784
	b813
	b1107
	b761
	b980
	b1012
	b765
b817
	b1095
	b1094
	b1093
	b1092
	b1091
	b1090
	b1075
	b1074
	b1070
	b1055
	b1072
	b815
	b1088
	b767
	b1081
	b1084
	b1085


	b288
	b284
b922
	b990
	b992
	b993
b1646
	b1642
	b1635
b308
	b360
	b346
	b326
	b318
	b293
	b315
	b306
b837
	b858
b1394
	b1400
	b1214
b132
	b98
	b106
	b105
	b121
	b122
	b125
	b127
b1294
	b1134
	b1310
b397
	b396
	b395
	b52
	b375
	b376
	b354
	b355
	b288
	b389
	b359
	b412
b806
	b741
	b1028
b1460
	b1714
	b1611
	b1740
	b1622
	b1623
	b1619
b521
	b652
b977
	b1001
	b988
	b898
	b847
b894
	b895
	b869
	b923
	b884
b526
	b576
	b529
	b495
b928
	b673
	b510
	b610
	b934
	b935
b836
	b835
b1073
	b1036
	b1050
	b1044
	b1023
	b1047
	b1042
	b1018
b1503
	b1549
	b1527
	b1513
	b1535
	b1732
	b1658
	b1528
	b1508
b1330
	b1323
	b1306
b949
	b951
	b904
b340
	b339
	b314
	b341
	b310
b1107
	b678
	b759
	b784
	b813
	b761
	b980
	b765
b209
	b207
	b224
	b232
	b222
	b221
b391
	b101
	b383
	b373
b1743
	b965
	b959
	b956
	b960
	b916
	b588
b993
	b922
b881
	b857
	b1057
	b546
	b848
	b849
	b897
	b844
	b1011
b425
	b542
	b710
	b714
b135
	b200
b1034
	b136
b628
	b631
	b545
b1395
	b1390
	b1392
	b1393
	b1399
	b1403
b880
	b1057
	b900
	b546
	

	b1544
b628
	b631
	b545
b1421
	b1497
	b1581
	b1425
	b1422
	b1419
	b1406
b115
	b73
	b75
	b81
	b80
	b83
	b78
	b283
b1603
	b1599
	b1614
	b1602
	b1610
	b1613
	b1612
	b1740
	b1623
	b1601
b1529
	b1579
	b1639
	b1564
b1213
	b1237
	b595
b1534
	b1596
	b1556
b346
	b394
	b360
	b308
	b293
	b315
	b307
	b306
b1386
	b1448
b1066
	b1095
	b1094
	b1093
	b1092
	b1091
	b1090
	b1075
	b1074
	b1070
	b1055
	b1072
	b815
	b817
	b767
	b1081
	b1084
	b1085
	b1086
	b1069
	b1088
	b1089
	b1065
	b1062
	b1063
	b1061
	b1054
	b798
	b1117
	b1037
b1467
	b1418
	b1690
	b1510
b904
	b949
b389
	b396
	b397
	b359
	b52
	b375
	b354
	b345
b1114
	b508
	b1050
	b1031
	b1023
	b1116
	b1024
	b1018
	b1017
	b1015
	b1048
	b1025
b576
	b532
	b533
	b530
	b531
	b526
	b529
	b656
b1599
	b1705
	b1618
	b1614
	b1616
	b1603
	b1613
	b1601
	b1620
	b1623
b1063
	b1095
	b1094
	b1093
	b1092
	b1091
	b1090
	b1075
	b1074
	b1070
	b1055
	b1072
	b815
	b1088
	b817
	b767
	b1081
	b1084
	b1085
	b1086
	b1069
	b1066
	b1089
	b1065
	b1062
	b1061
	b1054
	b798
	b1117
	b1037


	b751
b954
	b1003
	b957
b653
	b667
	b663
	b660
	b661
	b659
	b658
	b612
	b488
	b572
	b528
b554
	b635
b327
	b133
	b394
	b399
	b307
	b280
	b282
	b290
	b278
	b286
	b287
b1135
	b1165
	b1143
b162
	b455
	b246
	b166
b1205
	b1254
	b1211
	b1210
	b1206
	b1220
	b1342
b1130
	b1269
b515
	b735
	b655
	b746
b1103
	b755
	b795
b316
	b429
	b6
	b462
	b437
	b436
b1422
	b1497
	b1581
	b1425
	b1421
	b1419
	b1406
b873
	b872
	b874
	b883
	b871
	b870
	b877
	b876
	b867
	b889
	b930
b447
	b68
b189
	b170
	b185
	b259
	b197
	b85
	b178
b592
	b1030
	b1023
	b1036
	b943
	b1020
	b1018
	b1044
	b1045
	b1046
	b1043
	b1039
	b1029
	b1049
	b832
b1607
	b1608
	b1610
	b1679
b223
	b251
	b206
	b224
	b233
	b220
b1143
	b1200
	b1135
	b1165
b652
	b521
b1180
	b1222
	b1141
b1007
	b1002
	b1006
b384
	b322
b91
	b198
	b159
	b86
b643
	b651
b771
	b1080
	b1058
	b792
	b793
	b778
	b779
	b769
	b811
	b780
	b783
	b770
	b772
	b773
	b764
b1039
	b1036
	b1023
	b1050
	b943
	b832
	b1018
	b1044
	b1045
	b1046
	b1047
	b1035
	b592
	b1043
	b1038
	b1049
b1054
	b109

	b872
	b871
	b870
	b876
	b874
	b889
	b930
b107
	b98
	b161
	b126
	b122
	b127
	b131
b1528
	b1549
	b1513
	b1503
	b1732
	b1658
	b1508
b1195
	b1169
	b1198
	b1190
	b1197
b328
	b213
	b290
	b363
b26
	b45
	b27
	b285
b339
	b314
	b323
	b341
	b340
	b273
	b301
	b303
	b305
	b296
	b310
b1463
	b1469
	b1458
b149
	b146
	b180
	b150
	b173
	b179
b1298
	b1336
	b1295
	b1338
	b1249
	b1136
b1593
	b1625
	b1592
	b1574
b1146
	b1219
	b1413
b1619
	b1602
	b1611
	b1612
	b1627
	b1622
	b1623
	b1740
	b1460
b616
	b674
	b697
	b685
	b680
	b1112
	b679
	b615
b355
	b397
	b395
	b52
	b376
	b288
	b412
	b284
b1563
	b1683
	b1689
b1715
	b1716
	b1718
	b1721
	b1720
	b1722
	b1724
b833
	b763
	b897
	b900
	b952
b1502
	b1588
	b1496
	b1495
	b1514
	b1488
	b1489
b815
	b1095
	b1094
	b1093
	b1092
	b1091
	b1090
	b1075
	b1074
	b1070
	b1055
	b1072
	b1088
	b817
	b767
	b1081
	b1084
	b1085
	b1086
	b1069
	b1066
	b1089
	b1065
	b1062
	b1063
	b1061
	b1054
	b798
	b1117
	b1037
b1487
	b1443
	b1598
b287
	b399
	b327
	b133
	b280
	b281
	b282
	b290
	b279
	b278
